# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 63 new papers today
          18 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/17 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2211.03791


extracting tarball to tmp_2211.03791...

 done.
Retrieving document from  https://arxiv.org/e-print/2211.03796


extracting tarball to tmp_2211.03796...

...

extracting tarball to tmp_2211.04387...

 done.
Retrieving document from  https://arxiv.org/e-print/2211.04413


/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2211.04387/natnotes.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2211.04413...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.04048-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.04048) | **CAMEMBERT: A Mini-Neptunes GCM Intercomparison, Protocol Version 1.0. A  CUISINES Model Intercomparison Project**  |
|| <mark>Duncan A. Christie</mark>, et al. -- incl., <mark>Elspeth K. H. Lee</mark>, <mark>Linda Sohl</mark>, <mark>Maria E. Steinrueck</mark> |
|*Appeared on*| *2022-11-09*|
|*Comments*| *Accepted to PSJ*|
|**Abstract**| With an increased focus on the observing and modelling of mini-Neptunes, there comes a need to better understand the tools we use to model their atmospheres. In this paper, we present the protocol for the CAMEMBERT (Comparing Atmospheric Models of Extrasolar Mini-neptunes Building and Envisioning Retrievals and Transits) project, an intercomparison of general circulation models (GCMs) used by the exoplanetary science community to simulate the atmospheres of mini-Neptunes. We focus on two targets well studied both observationally and theoretically with planned JWST Cycle 1 observations: the warm GJ~1214b and the cooler K2-18b. For each target, we consider a temperature-forced case, a clear sky dual-grey radiative transfer case, and a clear sky multi band radiative transfer case, covering a range of complexities and configurations where we know differences exist between GCMs in the literature. This paper presents all the details necessary to participate in the intercomparison, with the intention of presenting the results in future papers. Currently, there are eight GCMs participating (ExoCAM, Exo-FMS, FMS PCM, Generic PCM, MITgcm, RM-GCM, THOR, and the UM), and membership in the project remains open. Those interested in participating are invited to contact the authors. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.03801-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.03801) | **Once is an Instance, Twice is a Hobby: Multiple Optical and  Near-Infrared Changing-Look Events in NGC 5273**  |
|| J. M. M. Neustadt, et al. -- incl., <mark>T. deJaeger</mark>, <mark>M. E. Huber</mark> |
|*Appeared on*| *2022-11-09*|
|*Comments*| *18 pages, 13 figures, 4 tables, submitting to MNRAS*|
|**Abstract**| NGC 5273 is a known optical and X-ray variable AGN. We analyze new and archival IR, optical, UV, and X-ray data in order to characterize its long-term variability from 2000 to 2022. At least one changing-look event occurred between 2011 and 2014, when the AGN changed from a Type 1.8/1.9 Seyfert to a Type 1. It then faded considerably at all wavelengths, followed by a dramatic but slow increase in UV/optical brightness between 2021 and 2022. We propose that NGC 5273 underwent multiple changing-look events between 2000 and 2022 -- starting as a Type 1.8/1.9, NGC 5273 changes-look to a Type 1 only temporarily in 2002 and again in 2014, reverting back to a Type 1.8/1.9 by 2005 and 2017, respectively. In 2022, it is again a Type 1 Seyfert with optical and NIR broad emission lines. We characterize the changing-look events and their connection to the dynamic accretion and radiative processes in NGC 5273. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.04160-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.04160) | **A Neural Network Subgrid Model of the Early Stages of Planet Formation**  |
|| <mark>Thomas Pfeil</mark>, et al. -- incl., <mark>Hubert Klahr</mark> |
|*Appeared on*| *2022-11-09*|
|*Comments*| *6 pages, 4 figures, accepted at the Machine Learning and the Physical Sciences workshop, NeurIPS 2022*|
|**Abstract**| Planet formation is a multi-scale process in which the coagulation of $\mathrm{\mu m}$-sized dust grains in protoplanetary disks is strongly influenced by the hydrodynamic processes on scales of astronomical units ($\approx 1.5\times 10^8 \,\mathrm{km}$). Studies are therefore dependent on subgrid models to emulate the micro physics of dust coagulation on top of a large scale hydrodynamic simulation. Numerical simulations which include the relevant physical effects are complex and computationally expensive. Here, we present a fast and accurate learned effective model for dust coagulation, trained on data from high resolution numerical coagulation simulations. Our model captures details of the dust coagulation process that were so far not tractable with other dust coagulation prescriptions with similar computational efficiency. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.03947-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.03947) | **Final stage of merging binaries of supermassive black holes:  observational signatures**  |
|| Jian-Min Wang, Yu-Yang Songsheng, <mark>Yan-Rong Li</mark>, Pu Du |
|*Appeared on*| *2022-11-09*|
|*Comments*| *10 pages, 3 figures; accepted to MNRAS*|
|**Abstract**| There are increasing interests in binary supermassive black holes (SMBHs), but merging binaries with separations smaller than ~1 light days (~10^2 gravitational radii for 10^8 Msun), which are rapidly evolving under control of gravitational waves, are elusive in observations. In this paper, we discuss fates of mini-disks around component SMBHs for three regimes: 1) low rates (advection-dominated accretion flows: ADAFs); 2) intermediate rates; 3) super-Eddington accretion rates. Mini-disks with intermediate rates are undergoing evaporation through thermal conduction of hot corona forming a hybrid radial structure. When the binary orbital periods are shorter than sound propagation timescales of the evaporated mini-disks, a new instability, denoted as sound instability, arises because the disks will be highly twisted so that they are destroyed. We demonstrate a critical separation of A_{crit}~10^2 Rg from the sound instability of the mini-disks and the cavity is full of hot gas. For those binaries, component SMBHs are accreting with Bondi mode in the ADAF regime, showing periodic variations resulting from Doppler boosting effects in radio from the ADAFs due to orbital motion. In the mean while, the circumbinary disks (CBDs) are still not hot enough (ultraviolet deficit) to generate photons to ionize gas for broad emission lines. For slightly super-Eddington accretion of the CBDs, MgII line appears with decreases of UV deficit, and for intermediate super-Eddington Balmer lines appear, but CIV line never unless CBD accretion rates are extremely high. Moreover, if the CBDs are misaligned with the binary plane, it is then expected to have optical periodical variations with about ten times radio periods. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.04260-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.04260) | **PhotoRedshift-MML: a multimodal machine learning method for estimating  photometric redshifts of quasars**  |
|| Shuxin Hong, et al. -- incl., <mark>A-Li Luo</mark>, <mark>Wenyu Yang</mark> |
|*Appeared on*| *2022-11-09*|
|*Comments*| *10 pages, 8 figures, accepted for publication in MNRAS*|
|**Abstract**| We propose a Multimodal Machine Learning method for estimating the Photometric Redshifts of quasars (PhotoRedshift-MML for short), which has long been the subject of many investigations. Our method includes two main models, i.e. the feature transformation model by multimodal representation learning, and the photometric redshift estimation model by multimodal transfer learning. The prediction accuracy of the photometric redshift was significantly improved owing to the large amount of information offered by the generated spectral features learned from photometric data via the MML. A total of 415,930 quasars from Sloan Digital Sky Survey (SDSS) Data Release 17, with redshifts between 1 and 5, were screened for our experiments. We used |{\Delta}z| = |(z_phot-z_spec)/(1+z_spec)| to evaluate the redshift prediction and demonstrated a 4.04% increase in accuracy. With the help of the generated spectral features, the proportion of data with |{\Delta}z| < 0.1 can reach 84.45% of the total test samples, whereas it reaches 80.41% for single-modal photometric data. Moreover, the Root Mean Square (RMS) of |{\Delta}z| is shown to decreases from 0.1332 to 0.1235. Our method has the potential to be generalized to other astronomical data analyses such as galaxy classification and redshift prediction. The algorithm code can be found at https://github.com/HongShuxin/PhotoRedshift-MML . |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.04328-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.04328) | **Cosmic void exclusion models and their impact on the distance scale  measurements from large scale structure**  |
|| Andrei Variu, et al. -- incl., <mark>Cheng Zhao</mark> |
|*Appeared on*| *2022-11-09*|
|*Comments*| *18 pages, 24 figures*|
|**Abstract**| Baryonic Acoustic Oscillations (BAOs) studies based on the clustering of voids and matter tracers provide important constraints on cosmological parameters related to the expansion of the Universe. However, modelling the void exclusion effect is an important challenge for fully exploiting the potential of these kind of analyses. We thus develop two numerical methods to describe the clustering of cosmic voids. Neither model requires additional cosmological information beyond that assumed within the galaxy de-wiggled model. The models consist in power spectra whose performance we assess in comparison to a parabolic model on both Patchy boxes and light-cones. Moreover, we test their robustness against systematic effects and the reconstruction technique. The void model power spectra and the parabolic model with a fixed parameter provide strongly correlated values for the Alcock-Paczynski ($\alpha$) parameter, for boxes and light-cones likewise. The resulting $\alpha$ values - for all three models - are unbiased and their uncertainties are correctly estimated. However, the numerical models show less variation with the fitting range compared to the parabolic one. The Bayesian evidence suggests that the numerical techniques are often favoured compared to the parabolic model. Moreover, the void model power spectra computed on boxes can describe the void clustering from light-cones as well as from boxes. The same void model power spectra can be used for the study of pre- and post-reconstructed data-sets. Lastly, the two numerical techniques are resilient against the studied systematic effects. Consequently, using either of the two new void models, one can more robustly measure cosmological parameters. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.04386-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.04386) | **Quick-Look Pipeline Light Curves for 5.7 Million Stars Observed Over the  Second Year of TESS' First Extended Mission**  |
|| Michelle Kunimoto, et al. -- incl., <mark>Katharine Hesse</mark> |
|*Appeared on*| *2022-11-09*|
|*Comments*| *3 pages, 1 figure*|
|**Abstract**| We present High-Level Science Products (HLSPs) containing light curves from MIT's Quick-Look Pipeline (QLP) from the second year of TESS' first Extended Mission (Sectors 40 - 55; 2021 July - 2022 September). In total, 12.2 million per-sector light curves for 5.7 million unique stars were extracted from 10-minute cadence Full-Frame Images (FFIs) and are made available to the community. As in previous deliveries, QLP HLSPs include both raw and detrended flux time series for all observed stars brighter than TESS magnitude T = 13.5 mag. Starting in Sector 41, QLP also produces light curves for select fainter M dwarfs. QLP has provided the community with one of the largest sources of FFI-extracted light curves to date since the start of the TESS mission. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.04387-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.04387) | **QLP Data Release Notes 002: Improved Detrending Algorithm**  |
|| Michelle Kunimoto, et al. -- incl., <mark>Katharine Hesse</mark> |
|*Appeared on*| *2022-11-09*|
|*Comments*| *3 pages, 1 figure*|
|**Abstract**| Light curves feature many kinds of variability, including instrumental systematics, intrinsic stellar variability such as pulsations, and flux changes caused by transiting exoplanets or eclipsing binary stars. Detrending is a key pre-planet-search data processing step that aims to remove variability not due to transits. This data release note describes improvements to the Quick-Look Pipeline's detrending algorithm via the inclusion of quaternion data to remove short-timescale systematics. We describe updates to our procedure, intermediate data products outputted by the algorithm, and improvements to light curve precision. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2211.04048.md
    + _build/html/tmp_2211.04048/./initial_pt_profiles.png
    + _build/html/tmp_2211.04048/./chem_profiles.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# 

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.04048-b31b1b.svg)](https://arxiv.org/abs/2211.04048)<mark>Appeared on: 2022-11-09</mark> - _Accepted to PSJ_

</div>
<div id="authors">

<mark>Duncan A. Christie</mark>, et al. -- incl., <mark>Elspeth K. H. Lee</mark>, <mark>Linda Sohl</mark>, <mark>Maria E. Steinrueck</mark>

</div>
<div id="abstract">

**Abstract:** With an increased focus on the observing and modelling of mini-Neptunes, there comes a need to better understand the tools we use to model their atmospheres.  In this paper, we present the protocol for the CAMEMBERT (Comparing Atmospheric Models of Extrasolar Mini-neptunes Building and Envisioning Retrievals and Transits) project, an intercomparison of general circulation models (GCMs) used by the exoplanetary science community to simulate the atmospheres of mini-Neptunes.   We focus on two targets well studied both observationally and theoretically with planned JWST Cycle 1 observations:  the warm GJ 1214b and the cooler K2-18b.  For each target, we consider a temperature-forced case, a clear sky dual-grey radiative transfer case, and a clear sky multi band radiative transfer case, covering a range of complexities and configurations where we know differences exist between GCMs in the literature.   This paper presents all the details necessary to participate in the intercomparison, with the intention of presenting the results in future papers.   Currently, there are eight GCMs participating ({\sc ExoCAM},{\sc Exo-FMS}, FMS PCM,{\sc Generic PCM},{\sc MITgcm}, RM-GCM, THOR, and the UM), and membership in the project remains open.  Those interested in participating are invited to contact the authors.

</div>

<div id="div_fig1">

<img src="tmp_2211.04048/./initial_pt_profiles.png" alt="Fig1" width="100%"/>

**Figure 1. -** Initial temperature profiles for K2-18b and GJ 1214b.  The solid lines indicate the initial temperature profiles for each planet while the shaded regions indicate the range of equilibrium temperatures $T_\mathrm{eq}$ in the temperature-forced case (see Equation \ref{Eqn:Teq}). \label{Fig:Profiles} (*Fig:Profiles*)

</div>
<div id="div_fig2">

<img src="tmp_2211.04048/./chem_profiles.png" alt="Fig2" width="100%"/>

**Figure 2. -** Chemical abundance profiles for K2-18b and GJ 1214b\label{Fig:ChemProfiles} to be used in Case 3.  These are publicly available in the CAMEMBERT repository (see Section \ref{Sec:Repo}). (*Fig:ChemProfiles*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

84  publications in the last 7 days.
	 _build/html/2211.04048.md
	 _build/html/2211.03641.md
	 _build/html/2211.02502.md
	 _build/html/2211.01474.md
	 _build/html/2211.00667.md
	 _build/html/2211.00657.md
	 _build/html/2211.00087.md
	 _build/html/2211.00036.md
	 _build/html/2211.00020.md
	 _build/html/2211.00010.md
	 _build/html/2210.17434.md
	 _build/html/2210.12412.md
	 _build/html/2210.11926.md
	 _build/html/2210.11551.md
	 _build/html/2210.11103.md
	 _build/html/2210.10707.md
	 _build/html/2210.09930.md
	 _build/html/2210.08996.md
	 _build/html/2210.08517.md
	 _build/html/2210.08510.md
	 _build/html/2210.08078.md
	 _build/html/2210.06504.md
	 _build/html/2210.05695.md
	 _build/html/2210.05595.md
	 _build/html/2210.05505.md
	 _build/html/2210.04721.md
	 _build/html/2210.03615.md
	 _build/html/2210.01811.md
	 _build/html/2210.01809.md
	 _build/html/2210.01556.md
	 _build/html/2210.00863.md
	 _build/html/2210.00560.md
	 _build/html/2209.15403.md
	 _build/html/2209.15110.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers